## Dependencies

In [1]:
import json
from pathlib import Path
import threading
import re
from typing import List, Iterable, Dict, Union, Optional

from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from datetime import datetime
import threading
import time
from collections import defaultdict

C:\Users\Aman Sheikh\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Storage directory configurations



In [2]:
STORAGE_DIR = Path.cwd().parent.parent / "storage"
STORAGE_DIR.mkdir(exist_ok=True)

SEED_DIR=STORAGE_DIR / "seeds"
SEED_DIR

WindowsPath('C:/Users/Aman Sheikh/Desktop/Projects/VeriFact/Model/harvester/storage/seeds')

## Helpers

In [3]:
_model_lock = threading.Lock()

In [4]:
class EmbedStats:
    def __init__(self, total_seeds: int):
        self.start = time.time()
        self.total_seeds = total_seeds
        self.seeds_done = 0
        self.phrases_embedded = 0
        self.lock = threading.Lock()

    def log_seed(self, n_phrases: int):
        with self.lock:
            self.seeds_done += 1
            self.phrases_embedded += n_phrases
            if self.seeds_done % 50 == 0 or self.seeds_done == self.total_seeds:
                elapsed = time.time() - self.start
                rate = self.phrases_embedded / max(elapsed, 1)
                print(
                    f"[STATS] seeds={self.seeds_done}/{self.total_seeds} | "
                    f"phrases={self.phrases_embedded} | "
                    f"{rate:.1f} phrases/sec | "
                    f"{elapsed:.1f}s elapsed"
                )

## Importing generated seeds

In [5]:
def import_seeds_for_harvest(
    path: Union[str, Path],
    branches: List[str] = None,
    strict_schema: bool = False
) -> List[Dict]:
    """
    Imports MeSH seeds for the harvester.

    Supports:
    - Single JSON file
    - Directory containing mesh_seed_<BRANCH>.json files

    Args:
        path: File or directory path
        branches: Optional list of branch letters to load (e.g. ['C', 'D'])
        strict_schema: If True, enforces minimal schema validation

    Returns:
        List of seed dictionaries
    """

    base_path = Path(path)
    seeds = []

    if not base_path.exists():
        raise FileNotFoundError(f"❌ Path does not exist: {base_path}")

    # -----------------------------
    # Resolve files to load
    # -----------------------------
    if base_path.is_file():
        files = [base_path]

    elif base_path.is_dir():
        files = sorted(base_path.glob("mesh_seed_*.json"))
        if not files:
            raise FileNotFoundError(f"❌ No mesh_seed_*.json files found in {base_path}")

    else:
        raise ValueError(f"❌ Invalid path: {base_path}")

    # -----------------------------
    # Load files
    # -----------------------------
    for file_path in files:
        # Branch inference from filename: mesh_seed_C.json
        match = re.match(r"mesh_seed_([A-Z]).json", file_path.name)
        file_branch = match.group(1) if match else None

        if branches and file_branch not in branches:
            continue

        try:
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)

            if not isinstance(data, list):
                raise ValueError("JSON root must be a list")

            for seed in data:
                if strict_schema:
                    _validate_seed_schema(seed)

                seeds.append(seed)

            print(f"📖 Loaded {len(data)} seeds from {file_path.name}")

        except json.JSONDecodeError:
            print(f"❌ Invalid JSON: {file_path}")
        except Exception as e:
            print(f"❌ Error loading {file_path.name}: {e}")

    print(f"✅ Total seeds loaded: {len(seeds)}")
    return seeds


In [6]:
def _validate_seed_schema(seed: Dict):
    required_keys = {
        "seed_id",
        "label",
        "canonical_label",
        "semantic_ground_truth",
        "temporal_tracking",
        "mesh_tree_numbers",
        "top_level_branches"
    }

    missing = required_keys - seed.keys()
    if missing:
        raise ValueError(f"Seed {seed.get('seed_id')} missing keys: {missing}")

    if not isinstance(seed["mesh_tree_numbers"], list):
        raise TypeError("mesh_tree_numbers must be a list")

    if not isinstance(seed["top_level_branches"], list):
        raise TypeError("top_level_branches must be a list")

## Extract seed keyword

```python
seed = {
    "label": "Cardiac Arrest",
    "synonyms": ["heart arrest", "cardiac arrest, heart stoppage", "SCA|sudden cardiac arrest"]
}

keywords = extract_seed_keywords(seed)
# -> ["cardiac arrest", "heart arrest", "heart stoppage", "sca", "sudden cardiac arrest"]
```

### Normalize keywords

In [7]:
def _normalize_keyword(s: str) -> str:
    """
    Normalize keyword for SapBERT:
    - lowercase
    - normalize separators
    - keep alphanumerics and spaces
    - collapse whitespace
    """
    if not s or not isinstance(s, str):
        return ""

    s = s.lower().strip()
    s = re.sub(r"[-_/]", " ", s)
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

### Split synonyms

In [8]:
def _split_synonym(s: str) -> Iterable[str]:
    """
    Split compound synonym strings safely.
    Handles:
      - commas
      - semicolons
      - pipes
    """
    if not s:
        return []
    return [p.strip() for p in re.split(r"[;,|]+", s) if p.strip()]

### Extract Keywords

In [9]:
def extract_seed_keywords_from_list(
    seeds: List[Dict],
    *,
    include_label: bool = True,
    include_synonyms: bool = True,
    max_phrase_tokens: int = 12,
    drop_label_duplicates: bool = True
) -> Dict[str, List[str]]:
    """
    Extract SapBERT keyword seeds from a list of MeSH-like seed dicts.

    Returns:
      {
        seed_id: [keyword1, keyword2, ...]
      }
    """

    results = {}

    for seed in seeds:
        seed_id = seed.get("seed_id", "unknown")
        label = seed.get("label", "")
        synonyms = seed.get("synonyms", [])

        seen = set()
        keywords = []

        def add_kw(phrase: str):
            norm = _normalize_keyword(phrase)
            if not norm:
                return

            # length guard (prevents huge IUPAC strings dominating embeddings)
            if len(norm.split()) > max_phrase_tokens:
                return

            if norm not in seen:
                seen.add(norm)
                keywords.append(norm)

        label_norm = _normalize_keyword(label)

        # --- label ---
        if include_label and label:
            add_kw(label)

        # --- synonyms ---
        if include_synonyms:
            for syn in synonyms:
                for part in _split_synonym(syn):
                    norm = _normalize_keyword(part)

                    if not norm:
                        continue

                    if drop_label_duplicates and norm == label_norm:
                        continue

                    add_kw(part)

        results[seed_id] = keywords
    print(f"✅ Extracted a total of {len(results)} seeds.")
    return results

## SapBERT configuration

In [10]:
# --- choose your SapBERT checkpoint (the one you use in your project) ---
MODEL_NAME = 'cambridgeltl/SapBERT-from-PubMedBERT-fulltext'  # <<< replace with your SapBERT model id

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

## SapBERT embedding

### Embedding Helpers

In [11]:
def mean_pooling(token_embeddings, attention_mask):
    """Mean pooling, ignoring padding tokens."""
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = (token_embeddings * input_mask_expanded).sum(1)
    sum_mask = input_mask_expanded.sum(1).clamp(min=1e-9)
    return sum_embeddings / sum_mask

In [12]:
def l2_normalize_np(arr: np.ndarray, eps: float = 1e-12) -> np.ndarray:
    norms = np.linalg.norm(arr, axis=1, keepdims=True)
    norms = np.maximum(norms, eps)
    return arr / norms

In [13]:
# # Thread-safe wrapper around embed_phrases_list (serializes model calls)
# def embed_phrases_list_threadsafe(phrases: List[str], batch_size: int = 32) -> np.ndarray:
#     """
#     Acquire a lock and call embed_phrases_list to avoid concurrent access to model/tokenizer.
#     embed_phrases_list should be the function that does tokenizer+model inference.
#     """
#     with _model_lock:
#         return embed_phrases_list(phrases, batch_size=batch_size)

In [14]:
# improved embed_phrases_list: fp16 when GPU available, returns l2-normalized float32 numpy array
def embed_phrases_list(phrases: List[str], tokenizer, model, device,
                       batch_size: int = 64, max_length: int = 64, normalize: bool = True) -> np.ndarray:
    """
    Embed a list of short phrases/terms (labels, synonyms, candidates).
    Returns shape (len(phrases), hidden_size) float32 numpy array, L2-normalized if normalize=True.
    """
    if not phrases:
        # return empty array with shape (0, hidden_size) if possible
        hidden = getattr(model.config, "hidden_size", None)
        if hidden is None:
            return np.zeros((0,), dtype=np.float32)
        return np.zeros((0, hidden), dtype=np.float32)

    model.eval()
    all_embs = []
    use_autocast = (device.type == "cuda")
    with torch.no_grad():
        for i in range(0, len(phrases), batch_size):
            batch = phrases[i:i+batch_size]
            enc = tokenizer(batch, padding=True, truncation=True, max_length=max_length, return_tensors="pt")
            input_ids = enc["input_ids"].to(device)
            attn = enc["attention_mask"].to(device)
            if use_autocast:
                # use automatic mixed precision on GPU for speed/memory
                with torch.cuda.amp.autocast():
                    out = model(input_ids=input_ids, attention_mask=attn, return_dict=True)
            else:
                out = model(input_ids=input_ids, attention_mask=attn, return_dict=True)

            token_embeds = out.last_hidden_state  # (batch, seq_len, hidden)
            pooled = mean_pooling(token_embeds, attn)  # (batch, hidden)
            emb = pooled.cpu().float().numpy()
            all_embs.append(emb)

    if not all_embs:
        return np.zeros((0, model.config.hidden_size), dtype=np.float32)

    arr = np.vstack(all_embs).astype(np.float32)
    if normalize and arr.size:
        arr = l2_normalize_np(arr)
    return arr

### Candidate generation
This will populate seed["keyword_candidates"] with scored candidates and sources.

In [15]:
def _clean_term(t: str) -> str:
    if not t: return ""
    t = t.strip()
    # remove excessive whitespace and repeated punctuation
    t = re.sub(r'\s+', ' ', t)
    return t

In [16]:
def generate_keyword_candidates(seed: Dict,
                                include_modifiers: bool = True,
                                max_modifier_per_seed: int = 10,
                                max_candidates: int = 200) -> List[Dict]:
    """
    Populate and return keyword_candidates for a seed based on canonical_label, label, synonyms, aliases and modifiers.
    Each candidate is dict: {"term": str, "score": float, "source": str}
    """
    seen = {}
    def add(term, score, source):
        if not term: return
        key = term.lower().strip()
        if not key: return
        if key in seen:
            # keep max score and add source
            seen[key]["score"] = max(seen[key]["score"], score)
            if source not in seen[key]["source"].split(";"):
                seen[key]["source"] += ";" + source
            return
        seen[key] = {"term": term, "score": float(score), "source": source}

    # 1) canonical_label (highest)
    add(seed.get("canonical_label", ""), 1.0, "canonical_label")
    # 2) label
    add(seed.get("label", ""), 0.98, "label")
    # 3) synonyms
    for s in seed.get("synonyms", []):
        add(_clean_term(s), 0.9, "mesh_synonym")
    # 4) aliases (lay terms)
    for a in seed.get("aliases", []):
        add(_clean_term(a), 0.85, "alias")
    # 5) preferred_search_terms (if present)
    for p in seed.get("preferred_search_terms", []):
        add(_clean_term(p), 0.95, "preferred_search")

    # 6) modifiers combos (term + modifier)
    if include_modifiers:
        modifiers = seed.get("harvester_modifiers", [])[:max_modifier_per_seed]
        base_terms = [seen[k]["term"] for k in list(seen.keys())][:3]  # use top 3 base terms
        for base in base_terms:
            for mod in modifiers:
                combo = f"{base} {mod}"
                add(_clean_term(combo), 0.6, f"modifier:{mod}")

    # 7) simple ngram reductions (2-3 token substrings)
    # yield token-level subsets for multiword terms
    for k, v in list(seen.items()):
        toks = v["term"].split()
        if len(toks) > 2:
            # take contiguous 2-gram and 3-gram windows
            for n in (2, 3):
                for i in range(len(toks) - n + 1):
                    add(" ".join(toks[i:i+n]), 0.5, "ngram")

    # produce ordered list by score
    sorted_candidates = sorted(seen.values(), key=lambda x: x["score"], reverse=True)[:max_candidates]
    seed["keyword_candidates"] = sorted_candidates
    # ---- preferred_search_terms (high-confidence only) ----
    preferred = [
        c["term"]
        for c in sorted_candidates
        if c["score"] >= 0.9 and c["source"] in {
            "canonical_label",
            "label",
            "mesh_synonym",
            "preferred_search"
        }
    ][:5]

    seed["preferred_search_terms"] = preferred
    return sorted_candidates

In [17]:
def _ensure_preferred_terms_from_candidates(seed: Dict, min_score: float = 0.90, max_terms: int = 5):
    """
    Ensure seed['preferred_search_terms'] is populated from keyword_candidates
    if not already present. Picks top scoring candidates with score >= min_score.
    """
    if seed.get("preferred_search_terms"):
        return
    cands = seed.get("keyword_candidates") or []
    # sort by score desc (they may already be sorted)
    cands_sorted = sorted(cands, key=lambda x: x.get("score", 0.0), reverse=True)
    preferred = []
    for c in cands_sorted:
        if c.get("score", 0.0) >= min_score:
            preferred.append(c.get("term"))
        if len(preferred) >= max_terms:
            break
    # fallback: take top N if nothing meets threshold
    if not preferred and cands_sorted:
        preferred = [c["term"] for c in cands_sorted[:min(max_terms, len(cands_sorted))]]
    seed["preferred_search_terms"] = preferred

### Embedding + persist per seed
This is the central function. It:

1. ensures candidates exist,

2. creates embedding directory structure per branch,

3. embeds canonical label and candidates,

4. saves labels.npy (1 x d) and candidates.npy (n x d),

5. updates seed['sapbert_metadata'] with refs and meta.

In [18]:
from typing import Union, List, Dict
from pathlib import Path
from datetime import datetime
import json
import numpy as np

def embed_and_persist_seed_embeddings(
    seeds: Union[Dict, List[Dict]],
    storage_root: Union[str, Path],
    tokenizer,
    model,
    device,
    batch_size: int = 64,
    max_length: int = 64,
    reembed: bool = False,
    stats: "EmbedStats" = None,
) -> Union[Dict, List[Dict]]:
    """
    Embed one seed dict or a list of seed dicts, persist .npy/.json files, and update seeds in-place.
    If `stats` is provided (EmbedStats), log progress per-seed.
    Returns updated seed or list of seeds (same shape as input).
    """
    single_input = isinstance(seeds, dict)
    seeds_list = [seeds] if single_input else list(seeds)
    storage_root = Path(storage_root)
    model_name = getattr(model.config, "name_or_path", None) or getattr(model, "base_model_prefix", "sapbert")

    updated_seeds = []
    for seed in seeds_list:
        # Ensure schema keys exist
        seed.setdefault("keyword_candidates", [])
        seed.setdefault("preferred_search_terms", [])
        seed.setdefault("excluded_terms", [])
        seed.setdefault("sapbert_metadata", {})

        # Ensure candidates and preferred terms
        if not seed.get("keyword_candidates"):
            generate_keyword_candidates(seed)
        _ensure_preferred_terms_from_candidates(seed)

        # Branch & uid logic
        branches = seed.get("top_level_branches") or ["X"]
        branch = branches[0] if isinstance(branches, (list, tuple)) else str(branches)
        uid = seed.get("seed_id", "").replace(":", "_").replace("/", "_")
        if not uid:
            uid = "mesh_unknown_" + str(abs(hash(seed.get("label", ""))))[:8]

        out_dir = storage_root / "embeddings" / branch
        out_dir.mkdir(parents=True, exist_ok=True)

        # Build candidate terms (canonical first)
        candidates = [c["term"] for c in seed.get("keyword_candidates", [])]
        canonical = seed.get("canonical_label") or seed.get("label")
        if canonical:
            candidates = [c for c in candidates if c.lower() != canonical.lower()]
            candidates.insert(0, canonical)

        if not candidates:
            print(f"[WARN] seed {uid} has no candidates, skipping embedding")
            if stats:
                stats.log_seed(0)
            updated_seeds.append(seed)
            continue

        # File paths
        labels_path = out_dir / f"{uid}_label.npy"
        cand_path = out_dir / f"{uid}_candidates.npy"
        meta_path = out_dir / f"{uid}_sapbert_meta.json"

        # Reuse existing embeddings if present and not reembed
        if not reembed and labels_path.exists() and cand_path.exists():
            try:
                lab = np.load(labels_path, mmap_mode="r")
                cands = np.load(cand_path, mmap_mode="r")
                seed.setdefault("sapbert_metadata", {})
                seed["sapbert_metadata"].update({
                    "model_name": model_name,
                    "label_embedding_ref": str(labels_path),
                    "candidate_embeddings_ref": str(cand_path),
                    "embedding_dim": int(lab.shape[1]) if lab.ndim > 1 else int(lab.shape[0]),
                    "num_candidate_embeddings": int(cands.shape[0]),
                    "embedded_at": datetime.utcnow().isoformat() + "Z"
                })
                total_phrases = 1 + int(cands.shape[0])
                print(f"[INFO] seed {uid}: reused existing embeddings ({total_phrases} phrases)")
                if stats:
                    stats.log_seed(total_phrases)
                updated_seeds.append(seed)
                continue
            except Exception as e:
                print(f"[WARN] seed {uid}: failed to read existing embeddings ({e}); will re-embed.")

        # Perform embedding
        try:
            all_embs = embed_phrases_list(
                candidates,
                tokenizer=tokenizer,
                model=model,
                device=device,
                batch_size=batch_size,
                max_length=max_length,
                normalize=True
            )
        except Exception as e:
            print(f"[ERROR] embedding failed for seed {uid}: {e}")
            if stats:
                stats.log_seed(0)
            updated_seeds.append(seed)
            continue

        # Validate embeddings
        if all_embs.shape[0] < 1:
            print(f"[WARN] no embeddings produced for seed {uid}, skipping.")
            if stats:
                stats.log_seed(0)
            updated_seeds.append(seed)
            continue

        label_emb = all_embs[0:1].astype(np.float32)
        cand_embs = all_embs[1:].astype(np.float32) if all_embs.shape[0] > 1 else np.zeros((0, all_embs.shape[1]), dtype=np.float32)

        # Persist arrays
        np.save(labels_path, label_emb)
        np.save(cand_path, cand_embs)

        # Persist metadata JSON
        meta = {
            "model_name": model_name,
            "label_embedding_ref": str(labels_path),
            "candidate_embeddings_ref": str(cand_path),
            "embedding_dim": int(label_emb.shape[1]),
            "num_candidate_embeddings": int(cand_embs.shape[0]),
            "candidates_counted": len(candidates),
            "embedded_at": datetime.utcnow().isoformat() + "Z"
        }
        try:
            with open(meta_path, "w", encoding="utf-8") as fh:
                json.dump(meta, fh, indent=2, ensure_ascii=False)
        except Exception as e:
            print(f"[WARN] failed to write metadata for {uid}: {e}")

        # Update seed metadata
        seed.setdefault("sapbert_metadata", {})
        seed["sapbert_metadata"].update({
            "model_name": model_name,
            "label_embedding_ref": str(labels_path),
            "candidate_embeddings_ref": str(cand_path),
            "embedding_dim": meta["embedding_dim"],
            "num_candidate_embeddings": meta["num_candidate_embeddings"],
            "embedded_at": meta["embedded_at"]
        })

        # Populate excluded_terms automatically if empty
        if not seed.get("excluded_terms"):
            exclusions = []
            for t in candidates:
                t_stripped = t.strip()
                if len(t_stripped) <= 1 or t_stripped.isdigit():
                    exclusions.append(t_stripped)
            seed["excluded_terms"] = list(dict.fromkeys(exclusions)) if exclusions else []

        # Ensure preferred_search_terms present (lower threshold fallback)
        if not seed.get("preferred_search_terms"):
            _ensure_preferred_terms_from_candidates(seed, min_score=0.75, max_terms=5)

        # Log stats & info
        num_phrases = len(candidates)
        print(f"[INFO] seed {uid}: embedded {num_phrases} phrases, saved to {out_dir}")
        if stats:
            stats.log_seed(num_phrases)

        updated_seeds.append(seed)

    # Return in same shape as input
    if single_input:
        return updated_seeds[0] if updated_seeds else seeds
    return updated_seeds

### Batch processor for many seeds

In [19]:
# from concurrent.futures import ThreadPoolExecutor, as_completed
#
# def prepare_seed_job(seed: Dict) -> Dict:
#     """
#     CPU-only preparation: candidate generation + term extraction.
#     """
#     if not seed.get("keyword_candidates"):
#         generate_keyword_candidates(seed)
#
#     canonical = seed.get("canonical_label") or seed.get("label")
#     terms = [canonical] + [c["term"] for c in seed["keyword_candidates"]]
#     # de-dup but preserve order
#     seen = set()
#     terms = [t for t in terms if not (t.lower() in seen or seen.add(t.lower()))]
#
#     return {
#         "seed": seed,
#         "terms": terms
#     }

In [20]:
# def embed_jobs_batched(
#     jobs: list,
#     storage_root: Path,
#     tokenizer,
#     model,
#     device,
#     batch_size: int,
#     max_length: int,
#     stats: EmbedStats,
#     reembed: bool = False
# ):
#     """
#     Embed many seeds in large batches on CPU.
#     """
#     for job in jobs:
#         seed = job["seed"]
#         terms = job["terms"]
#
#         uid = seed["seed_id"].replace(":", "_")
#         branch = seed["top_level_branches"][0]
#         out_dir = storage_root / "embeddings" / branch
#         out_dir.mkdir(parents=True, exist_ok=True)
#
#         label_path = out_dir / f"{uid}_label.npy"
#         cand_path = out_dir / f"{uid}_candidates.npy"
#
#         if not reembed and label_path.exists() and cand_path.exists():
#             stats.log_seed(0)
#             continue
#
#         emb = embed_phrases_list(
#             terms,
#             tokenizer=tokenizer,
#             model=model,
#             device=device,
#             batch_size=batch_size,
#             max_length=max_length,
#             normalize=True
#         )
#
#         np.save(label_path, emb[:1])
#         np.save(cand_path, emb[1:])
#
#         seed.setdefault("sapbert_metadata", {}).update({
#             "model_name": model.config.name_or_path,
#             "label_embedding_ref": str(label_path),
#             "candidate_embeddings_ref": str(cand_path),
#             "embedding_dim": emb.shape[1],
#             "num_candidate_embeddings": emb.shape[0] - 1,
#             "embedded_at": time.strftime("%Y-%m-%dT%H:%M:%SZ")
#         })
#
#         stats.log_seed(len(terms))

In [21]:
# def process_seeds_batch(
#     seeds: list,
#     storage_root: Union[str, Path],
#     tokenizer,
#     model,
#     device,
#     prep_workers: int = 8,
#     embed_batch_seeds: int = 64,
#     embed_batch_size: int = 128,
#     max_length: int = 64,
#     reembed: bool = False
# ):
#     """
#     High-throughput CPU-safe pipeline:
#     - multithreaded preparation
#     - single-threaded batched embedding
#     """
#     storage_root = Path(storage_root)
#     stats = EmbedStats(total_seeds=len(seeds))
#
#     # 1️⃣ Prepare jobs in parallel
#     print("🔧 Preparing seeds (multithreaded)...")
#     jobs = []
#     with ThreadPoolExecutor(max_workers=prep_workers) as ex:
#         futures = [ex.submit(prepare_seed_job, s) for s in seeds]
#         for f in as_completed(futures):
#             jobs.append(f.result())
#
#     print(f"✅ Prepared {len(jobs)} seeds. Starting embedding...")
#
#     # 2️⃣ Embed in large batches (single thread)
#     for i in range(0, len(jobs), embed_batch_seeds):
#         batch = jobs[i:i + embed_batch_seeds]
#         embed_jobs_batched(
#             jobs=batch,
#             storage_root=storage_root,
#             tokenizer=tokenizer,
#             model=model,
#             device=device,
#             batch_size=embed_batch_size,
#             max_length=max_length,
#             stats=stats,
#             reembed=reembed
#         )
#
#     print("🎉 Embedding completed.")
#     return seeds

## Export the generated keyword dictionary

In [22]:
def save_seeds_by_mesh_branch(
    seeds: List[Dict],
    storage_path: Union[str, Path],
    filename_prefix: str = "mesh_seed"
) -> Dict[str, Path]:
    """
    Saves seeds into separate JSON files per top-level MeSH branch.
    Example:
        mesh_seed_C.json
        mesh_seed_D.json
        mesh_seed_G.json

    Returns:
        Dict mapping branch letter -> file path
    """

    base_dir = Path(storage_path)
    base_dir.mkdir(parents=True, exist_ok=True)

    # Group seeds by branch
    branch_buckets = defaultdict(list)

    for seed in seeds:
        for branch in seed.get("top_level_branches", []):
            branch_buckets[branch].append(seed)

    saved_files = {}

    for branch, branch_seeds in branch_buckets.items():
        file_path = base_dir / f"{filename_prefix}_{branch}.json"

        try:
            with open(file_path, "w", encoding="utf-8") as f:
                json.dump(branch_seeds, f, indent=4, ensure_ascii=False)

            print(f"✅ Saved {len(branch_seeds)} seeds → {file_path}")
            saved_files[branch] = file_path

        except Exception as e:
            print(f"❌ Error saving branch {branch}: {e}")

    return saved_files

In [23]:
# def export_keywords_to_json(
#     keyword_map: Dict[str, List[str]],
#     output_path: str,
#     *,
#     indent: int = 2,
#     sort_keys: bool = True
# ) -> None:
#     """
#     Export extracted SapBERT seed keywords to a JSON file.
#
#     Args:
#       keyword_map: {seed_id: [keyword1, keyword2, ...]}
#       output_path: path to output JSON file
#       indent: JSON indentation level (default=2)
#       sort_keys: sort dictionary keys for reproducibility
#     """
#
#     path = Path(output_path)
#     path.parent.mkdir(parents=True, exist_ok=True)
#
#     with path.open("w", encoding="utf-8") as f:
#         json.dump(
#             keyword_map,
#             f,
#             ensure_ascii=False,
#             indent=indent,
#             sort_keys=sort_keys
#         )
#     print(f"✅ File saved to {path} successfully.")

## Example usage

#### Importing the data

In [24]:
SEED_FILE = SEED_DIR / "seeds_mesh.json"

# Load everything (recommended default)
seeds=import_seeds_for_harvest(SEED_DIR)

# # Load only disease & drug seeds
# seeds = import_seeds_for_harvest(
#     "storage/seeds",
#     branches=["C", "D"]
# )
# # Load a single branch file
# seeds = import_seeds_for_harvest(
#     "storage/seeds/mesh_seed_C.json"
# )
# # Enforce schema during development
# seeds = import_seeds_for_harvest(
#     "storage/seeds",
#     strict_schema=True
# )

📖 Loaded 114 seeds from mesh_seed_A.json
📖 Loaded 22 seeds from mesh_seed_B.json
📖 Loaded 5069 seeds from mesh_seed_C.json
📖 Loaded 10688 seeds from mesh_seed_D.json
📖 Loaded 2467 seeds from mesh_seed_E.json
📖 Loaded 974 seeds from mesh_seed_F.json
📖 Loaded 2513 seeds from mesh_seed_G.json
📖 Loaded 139 seeds from mesh_seed_H.json
📖 Loaded 229 seeds from mesh_seed_I.json
📖 Loaded 363 seeds from mesh_seed_J.json
📖 Loaded 23 seeds from mesh_seed_K.json
📖 Loaded 143 seeds from mesh_seed_L.json
📖 Loaded 361 seeds from mesh_seed_M.json
📖 Loaded 1400 seeds from mesh_seed_N.json
📖 Loaded 3 seeds from mesh_seed_Z.json
✅ Total seeds loaded: 24508


#### Extract keyword + embed and persist

In [25]:
# generate keywords + embed and persist
# results = process_seeds_batch(seeds, storage_root="storage", tokenizer=tokenizer, model=model, device=device)
results = embed_and_persist_seed_embeddings(seeds, storage_root="storage", tokenizer=tokenizer, model=model, device=device)
# keywords = extract_seed_keywords_from_list(seed_data)
# keywords
# results = embed_and_score(keywords, batch_size=32, max_workers=8)

[INFO] seed mesh_D000153: reused existing embeddings (146 phrases)
[INFO] seed mesh_D000961: reused existing embeddings (142 phrases)
[INFO] seed mesh_D002503: reused existing embeddings (25 phrases)


C:\Users\Aman Sheikh\AppData\Local\Temp\ipykernel_7944\4135559488.py:82: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "embedded_at": datetime.utcnow().isoformat() + "Z"
C:\Users\Aman Sheikh\AppData\Local\Temp\ipykernel_7944\4135559488.py:134: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "embedded_at": datetime.utcnow().isoformat() + "Z"


[INFO] seed mesh_D002523: embedded 102 phrases, saved to storage\embeddings\G
[INFO] seed mesh_D002842: reused existing embeddings (25 phrases)
[INFO] seed mesh_D002843: reused existing embeddings (32 phrases)
[INFO] seed mesh_D002875: reused existing embeddings (28 phrases)
[INFO] seed mesh_D002876: reused existing embeddings (66 phrases)
[INFO] seed mesh_D002877: reused existing embeddings (78 phrases)
[INFO] seed mesh_D002878: reused existing embeddings (53 phrases)
[INFO] seed mesh_D002879: reused existing embeddings (53 phrases)
[INFO] seed mesh_D002880: reused existing embeddings (53 phrases)
[INFO] seed mesh_D002881: reused existing embeddings (53 phrases)
[INFO] seed mesh_D002882: reused existing embeddings (53 phrases)
[INFO] seed mesh_D002883: reused existing embeddings (53 phrases)
[INFO] seed mesh_D002884: reused existing embeddings (53 phrases)
[INFO] seed mesh_D002885: reused existing embeddings (53 phrases)
[INFO] seed mesh_D002886: reused existing embeddings (53 phrases

In [26]:
# print(results)

#### Save the seeds by MeSH Branches

In [27]:
# save updated seeds back to per-branch files (use save_seeds_by_mesh_branch)
save_seeds_by_mesh_branch(results, SEED_DIR)

✅ Saved 5436 seeds → C:\Users\Aman Sheikh\Desktop\Projects\VeriFact\Model\harvester\storage\seeds\mesh_seed_C.json
✅ Saved 261 seeds → C:\Users\Aman Sheikh\Desktop\Projects\VeriFact\Model\harvester\storage\seeds\mesh_seed_A.json
✅ Saved 11084 seeds → C:\Users\Aman Sheikh\Desktop\Projects\VeriFact\Model\harvester\storage\seeds\mesh_seed_D.json
✅ Saved 3609 seeds → C:\Users\Aman Sheikh\Desktop\Projects\VeriFact\Model\harvester\storage\seeds\mesh_seed_G.json
✅ Saved 821 seeds → C:\Users\Aman Sheikh\Desktop\Projects\VeriFact\Model\harvester\storage\seeds\mesh_seed_J.json
✅ Saved 58 seeds → C:\Users\Aman Sheikh\Desktop\Projects\VeriFact\Model\harvester\storage\seeds\mesh_seed_B.json
✅ Saved 3410 seeds → C:\Users\Aman Sheikh\Desktop\Projects\VeriFact\Model\harvester\storage\seeds\mesh_seed_E.json
✅ Saved 2476 seeds → C:\Users\Aman Sheikh\Desktop\Projects\VeriFact\Model\harvester\storage\seeds\mesh_seed_N.json
✅ Saved 1568 seeds → C:\Users\Aman Sheikh\Desktop\Projects\VeriFact\Model\harvester

{'C': WindowsPath('C:/Users/Aman Sheikh/Desktop/Projects/VeriFact/Model/harvester/storage/seeds/mesh_seed_C.json'),
 'A': WindowsPath('C:/Users/Aman Sheikh/Desktop/Projects/VeriFact/Model/harvester/storage/seeds/mesh_seed_A.json'),
 'D': WindowsPath('C:/Users/Aman Sheikh/Desktop/Projects/VeriFact/Model/harvester/storage/seeds/mesh_seed_D.json'),
 'G': WindowsPath('C:/Users/Aman Sheikh/Desktop/Projects/VeriFact/Model/harvester/storage/seeds/mesh_seed_G.json'),
 'J': WindowsPath('C:/Users/Aman Sheikh/Desktop/Projects/VeriFact/Model/harvester/storage/seeds/mesh_seed_J.json'),
 'B': WindowsPath('C:/Users/Aman Sheikh/Desktop/Projects/VeriFact/Model/harvester/storage/seeds/mesh_seed_B.json'),
 'E': WindowsPath('C:/Users/Aman Sheikh/Desktop/Projects/VeriFact/Model/harvester/storage/seeds/mesh_seed_E.json'),
 'N': WindowsPath('C:/Users/Aman Sheikh/Desktop/Projects/VeriFact/Model/harvester/storage/seeds/mesh_seed_N.json'),
 'F': WindowsPath('C:/Users/Aman Sheikh/Desktop/Projects/VeriFact/Model/